In [22]:
!nvidia-smi

In [23]:
#from google.colab import drive
#drive.mount('/content/drive')

## Download the data:

## Unzip:

In [24]:
'''
!pip install emoji
!pip install transformers
!pip install ekphrasis
'''

In [25]:
import pandas as pd
import csv
import re
import emoji
import os
from datetime import datetime
import logging
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertForSequenceClassification, AdamW, BertConfig
from tqdm import tqdm, trange
import numpy as np
import time
import random
import datetime
from _datetime import datetime as dt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer, AutoModelWithLMHead, AutoConfig, AutoModelForSequenceClassification
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    get_linear_schedule_with_warmup
)
from sklearn import preprocessing

In [26]:
"""from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

def clean_tweet(tweet):
  return " ".join(text_processor.pre_process_doc(tweet))
"""

In [27]:
def clean_tweet(tweet):
    tweet = re.sub(r'#([^ ]*)', r'\1', tweet)
    tweet = re.sub(r'https.*[^ ]', 'URL', tweet)
    tweet = re.sub(r'http.*[^ ]', 'URL', tweet)
    tweet = emoji.demojize(tweet)
    tweet = re.sub(r'(:.*?:)', r' \1 ', tweet)
    tweet = re.sub(' +', ' ', tweet)
    return tweet

def get_data(path):
  tweets = []
  labels = []

  dataset = pd.read_csv(path, sep = ',', quotechar='"', usecols = ['id', 'text', 'hs'])
  for tweet in dataset["text"].tolist():
    tweets.append(tweet)
  labels = dataset["hd"].tolist()
  tweet_ids = dataset["id"].tolist()
  return tweets, labels, tweet_ids

In [28]:
import pandas as pd
def get_olid_train():
  tweets = []
  labels = []

  dataset = pd.read_csv("../input/dati-di-training-e-di-test/haspeede2_dev_taskAB.csv", sep = '\t',header=None)
  dataset=dataset.drop([0])
  for tweet in dataset[1].tolist():
    tweets.append(tweet)
  labels = dataset[2].tolist()
  tweet_ids = dataset[0].tolist()
  return tweets, labels, tweet_ids

def get_olid_test():
  testset = pd.read_csv("../input/dati-di-training-e-di-test/haspeede2_reference_taskAB-tweets.csv", sep = '\t',header=None)
  return testset[1].tolist(), testset[2].tolist(), testset[0].tolist()


In [29]:
def encode_tweets(tweets, tokenizer):

    input_ids = []
    max_length = 100


    for sent in tweets:
        encoded_sent = tokenizer.encode(
            sent,
            add_special_tokens = True,
            max_length = max_length)  # orignal value 512
        
        input_ids.append(encoded_sent)

    input_ids = pad_sequences(input_ids, maxlen = max_length, dtype = "long",
                                    value = tokenizer.pad_token_id, truncating = "pre", padding = "pre")

    return input_ids

In [30]:
def create_attention_masks(input_ids):
    attention_masks = []

    # For each tweet in the training set
    for sent in input_ids:
        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in sent]

        # Store the attention mask for this sentence.
        attention_masks.append(att_mask)
    return attention_masks

In [31]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [32]:
# Logger stuff
logFormatter = logging.Formatter("%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s")
logger = logging.getLogger(__name__)

In [33]:
# -----------------------------
# TYPE HERE THE MODEL NAME TO CHANGE IT
# -----------------------------

# model_name = "roberta-base"
model_name = "Musixmatch/umberto-commoncrawl-cased-v1"
# model_name = "microsoft/deberta-base-mnli"

# Returns a datetime object containing the local date and time (used for output_model_dir)
dateTimeObj = str(dt.now()).replace(" ", "_")


# Set the seed value all over the place to make this reproducible.
# seed_val = 42
# random.seed(seed_val)

seed_val = random.randint(0, 100)
print(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Directory in which the model will be saved along with the log
output_model_dir = './' + model_name.replace('/', '') + "/" + dateTimeObj + "_SEED_" + str(seed_val) +"/"

# Make dir for model serializations
os.makedirs(os.path.dirname(output_model_dir), exist_ok=True)

# Log stuff: print logger on file in output_model_dir/log.log
logging.basicConfig(filename=output_model_dir + 'log.log', level=logging.DEBUG)

# Log stuff: print logger also on stderr
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
logger.addHandler(consoleHandler)

In [34]:
# -----------------------------
# Load Pre-trained BERT model
# -----------------------------
config_class = AutoConfig
model_class = AutoModelForSequenceClassification
tokenizer_class = AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load a trained model and vocabulary pre-trained for specific language
logger.info("Loading model") #from: '" + model_dir + "', it may take a while...")

# Load pre-trained Tokenizer from directory, change this to load a tokenizer from ber package
tokenizer = tokenizer_class.from_pretrained(model_name)

# Load Bert for classification 'container'
model = model_class.from_pretrained(
     model_name, # Use pre-trained model from its directory, change this to use a pre-trained model from bert
     num_labels = 2, # The number of output labels--2 for binary classification.
                     # You can increase this for multi-class tasks.
     output_attentions = False, # Whether the model returns attentions weights.
     output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Set the model to work on CPU if no GPU is present
model.to(device)
logger.info("Bert for classification model has been loaded!")



In [35]:
# --------------------------------------------------------------------
# ---------- Print BERT model list of parameters and layers ----------
# --------------------------------------------------------------------
# The list of prints can be safely removed

# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

logger.info('The BERT model has {:} different named parameters.\n'.format(len(params)))

logger.info('==== Embedding Layer ====\n')

for p in params[0:5]:
    logger.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

logger.info('\n==== First Transformer ====\n')

for p in params[5:21]:
    logger.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

logger.info('\n==== Output Layer ====\n')

for p in params[-4:]:
    logger.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


In [36]:
train_path = "../input/dati-di-training-e-di-test/haspeede2_dev_taskAB.csv"

# Train set

# train_tweets, train_labs, _ = get_data(train_path)
train_tweets, train_labs, _ = get_olid_train()
train_ids = encode_tweets(train_tweets, tokenizer)
train_masks = create_attention_masks(train_ids)

train_labs=[int(x) for x in train_labs]


In [37]:
# --------------------------------------------------------------------
# -------------------- Split train and validation --------------------
# --------------------------------------------------------------------
#
# If the dataset is not partitioned into Train/Test we have to split it
# Split train and validation
# Use 90% for training and 10% for validation.

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_ids, train_labs, random_state=seed_val, test_size=0.1, stratify=train_labs)

# Do the same for the masks.

train_masks, validation_masks, _, _ = train_test_split(train_masks, train_labs, random_state=seed_val, test_size=0.1, stratify=train_labs)

# Convert all inputs and labels into torch tensors, the required datatype for our model.

# Tweets
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

# Labels
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

# Attention masks
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# We will use a DataLoader, it helps save on memory during training because, unlike a for loop, with an iterator
# the entire dataset does not need to be loaded into memory
# The DataLoader needs to know our batch size for training, so we specify it here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
# Note that the number of batch has to be the same, this means that we have to aggregate results in the end
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [38]:
# --------------------------------------------------------------------
# -------------- Optimizer and Learning Rate Scheduler ---------------
# --------------------------------------------------------------------
# For the purposes of fine-tuning, the authors recommend choosing from the following values:

# Batch size: 16, 32 (We chose 32 when creating our DataLoaders).
# Learning rate (Adam): 5e-5, 3e-5, 2e-5 (We’ll use 2e-5).
# Number of epochs: 2, 3, 4 (We’ll use 4).
#
#
# Note: AdamW is a class from the HuggingFace library (as opposed to PyTorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr=2e-4,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs (authors recommend between 2 and 4)
epochs = 10

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Default value in run_glue.py
                                            num_training_steps=total_steps)

In [ ]:
# --------------------------------------------------------------------
# Now we are ready to prepare and run the training/evaluation
# --------------------------------------------------------------------
#
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in tqdm(range(0, epochs), desc="Training"):

    # ========================================
    #               Training
    # ========================================

    # Store true lables for global eval
    gold_labels = []
    # Store  predicted labels for global eval
    predicted_labels = []

    # Perform one full pass over the training set.

    logger.info("")
    logger.info('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    logger.info('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    # the tqdm instruction mess with prints on terminal but it can be useful to understand what is the current
    # batch at any time
    for step, batch in tqdm(enumerate(train_dataloader), desc="Batch"):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            logger.info('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # The call to `model` always returns a tuple, so we need to pull the
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    logger.info("")
    logger.info("  Average training loss: {0:.2f}".format(avg_train_loss))
    logger.info("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ------------------------------------------------------------------------------------------------------------------
    # Todo: Cut code from here to remove the validation step: the loading function has to be changed in order
    #  to parse the training set only
    # ------------------------------------------------------------------------------------------------------------------

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    logger.info("")
    logger.info("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU/CPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()

       # Store gold labels single list
        gold_labels.extend(labels_flat)
        # Store predicted labels single list
        predicted_labels.extend(pred_flat)

        # The classification report is printed on the log, note that print one report for each validation epoch,
        # if we want to compute an average P/R/F1 we can do the same as accuracy, that is an accumulator that
        # stores P/R over epochs or compute the average at the end

        logger.info(classification_report(labels_flat,pred_flat, digits=4))

    # ------------------------------------------------------------------------------------------------------------------
    # Todo: Cut code until here to remove the validation step
    # ------------------------------------------------------------------------------------------------------------------

    # Report the final accuracy for this validation run.
    logger.info("  Accuracy: {0:.2f}".format(eval_accuracy / nb_eval_steps))
    logger.info("  Validation took: {:}".format(format_time(time.time() - t0)))


    logger.info("")
    logger.info("Evaluation on full prediction per epoch!")
    logger.info("Gold labels" + str(len(gold_labels)))
    logger.info("Predicted labels" + str(len(predicted_labels)))
    logger.info(classification_report(gold_labels,predicted_labels, digits=4))


logger.info("")
logger.info("Training complete!")

logger.info("Saving model to: " + output_model_dir)
logger.info("# Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()")
## Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
#
# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = (
    model.module if hasattr(model, "module") else model
)  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

In [ ]:
def classification_report_to_dataframe(str_representation_of_report):
    split_string = [x.split(' ') for x in str_representation_of_report.split('\n')]
    column_names = ['']+[x for x in split_string[0] if x!='']
    values = []
    for table_row in split_string[1:-1]:
        table_row = [value for value in table_row if value!='']
        if table_row!=[]:
            values.append(table_row)
    for i in values:
        for j in range(len(i)):
            if i[1] == 'avg':
                i[0:2] = [' '.join(i[0:2])]
            if len(i) == 3:
                i.insert(1,np.nan)
                i.insert(2, np.nan)
            else:
                pass
    report_to_df = pd.DataFrame(data=values, columns=column_names)
    return report_to_df

In [ ]:
# ---------------------------- Load Model ----------------------------

# Directory where the pretrained model can be found

# model_dir = '/content/drive/MyDrive/bert_models/roberta-base/2021-10-24_16:45:27.361306/'
model_dir = './Musixmatchumberto-commoncrawl-cased-v1/2021-11-11_18:02:18.375450_SEED_32'
#model_dir = '/content/drive/MyDrive/bert_models/microsoftdeberta-base-mnli/2021-10-30_13:14:31.630815/'
test_path = "../input/dati-di-training-e-di-test/haspeede2_reference_taskAB-tweets.csv"

output_file = model_dir + 'output.csv'
print(test_path)
print(output_file)
# Returns a datetime object containing the local date and time
dateTimeObj = str(dt.now()).replace(" ", "_")

# Log stuff: print logger on file
# Make dir for model serializations


logging.basicConfig(filename=model_dir + 'prediction_log', level=logging.DEBUG)

# Log stuff: print logger also on stderr
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
logger.addHandler(consoleHandler)

# -----------------------------
# Load Pre-trained BERT model
# -----------------------------
config_class = AutoConfig
model_class = AutoModelForSequenceClassification
tokenizer_class = AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load a trained model and vocabulary pre-trained for specific language
logger.info("Loading model from: '" + model_dir + "', it may take a while...")

# Load pre-trained Tokenizer from directory, change this to load a tokenizer from ber package
tokenizer = tokenizer_class.from_pretrained(model_dir)

# Load Bert for classification 'container'
model = model_class.from_pretrained(
    model_dir, # Use pre-trained model from its directory, change this to use a pre-trained model from bert
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Set the model to work on CPU if no GPU is present
model.to(device)
logger.info("Bert for classification model has been loaded!")

# ---------------------------- Predict ----------------------------
output_file = model_dir + 'output.csv'

# Returns a datetime object containing the local date and time
dateTimeObj = str(dt.now()).replace(" ", "_")


logging.basicConfig(filename=(model_dir + 'prediction_log') + '.log', level=logging.DEBUG)

# Log stuff: print logger also on stderr
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
logger.addHandler(consoleHandler)

# Set the model to work on CPU if no GPU is present
model.to(device)


# --------------------------------------------------------------------
# -------------------------- Load test data --------------------------
# --------------------------------------------------------------------

# test_tweets, test_labs, tweet_ids = get_data(test_path)
test_tweets, test_labs, tweet_ids = get_olid_test()
test_ids = encode_tweets(test_tweets, tokenizer)
test_masks = create_attention_masks(test_ids)


#print(tweet_labels)
# Tweets
prediction_inputs = torch.tensor(test_ids)

# Attention masks
prediction_masks = torch.tensor(test_masks)

label_encoder = preprocessing.LabelEncoder()
targets = label_encoder.fit_transform(tweet_ids)

prediction_ids = torch.as_tensor(targets)
# targets: tensor([0, 1, 2, 3])

# Set the batch size.
batch_size = 32

# Create the DataLoader.
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_ids)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables
predictions = []

# Predict
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_tweet_ids = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    flat_logits = np.argmax(logits, axis=1).flatten()
    # Get tweet ids for prediction output
    ids = label_encoder.inverse_transform(b_tweet_ids.cpu().numpy())
    # Store predictions and true labels
    predictions.extend(list(zip(ids, flat_logits)))

# Print the list of prediction & store for evaluation
with open(output_file, 'w') as out_file:
    # store predictions in list for eval
    pred_ = []
    # Get each tweet id
    for tweet_id_prediction in predictions:
        # Print the prediction todo: debug to remove
        #print(str(tweet_id_prediction[0]) + "\t" + str(tweet_id_prediction[1]))
        label = tweet_id_prediction[1]
        pred_.append(label)
        out_file.write(str(tweet_id_prediction[0]) + "," + str(label) + '\n')

logger.info(len(pred_))
logger.info(len(test_labs))

cr = classification_report(test_labs,pred_, digits=4)
logger.info(cr)
classification_report_to_dataframe(cr).to_csv(model_dir + "cr_report.csv")

print(cr)



print('    DONE.')

In [ ]:
'''
import os
import pandas as pd
ekpath = '/content/drive/MyDrive/bert_models/olid/5-seed-ernie/ekphrasis_preprocessing/nghuyongernie-2.0-en'
simpath = '/content/drive/MyDrive/bert_models/olid/5-seed-ernie/simple_preprocessing/nghuyongernie-2.0-en'

path = ekpath
csv = []
for i in os.listdir(path):
  csv.append(os.path.join(* [path, i, 'cr_report.csv']))
print(csv)

df = []
for i in csv:
  df.append(pd.read_csv(i))

precision = []
recall = []
f1_score = []
for i in df:
  precision.append(i['precision'])
  recall.append(i['recall'])
  f1_score.append(i['f1-score'])
  
precision = sum(precision)/len(df)
recall = sum(recall)/len(df)
f1_score = sum(f1_score)/len(f1_score)

avg_df = pd.DataFrame()
avg_df[' '] = df[0]['Unnamed: 1']
avg_df['precision'] = precision
avg_df['recall'] = recall
avg_df['f1-score'] = f1_score
avg_df.to_csv(path + '/avg_cr_report.csv')
'''